In [ ]:
import os
#import plaidmlkeras
# import plaidml.keras
# plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import sys

sys.path.insert(0, '../') #to load FileDataGenerator

import keras
import split_folders
from keras import losses
from keras import optimizers
from keras.regularizers import l2, l1
#kullback_leibler_divergence
#categorical_crossentropy
from keras.metrics import categorical_accuracy
#from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
#from keras.applications.vgg19 import VGG19, preprocess_input
#from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
#from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D, Dropout, GaussianNoise, Average, Minimum, Maximum
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import SGD, Adam, Adadelta, Nadam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.models import load_model as klm
import numpy as np
#from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import os.path
import fnmatch
import itertools
import seaborn
from itertools import chain
import functools
from glob import glob
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
from FileDataGenerator import FileDataGen
import PIL
#import losses as custom_losses
#from focal_loss import binary_focal_loss
import Augmentor
from livelossplot.keras import PlotLossesCallback
from keras.preprocessing.image import img_to_array
#from cycliclr import CyclicLR
from keras_radam import RAdam
from keras_lookahead import Lookahead
from medpy.filter.smoothing import anisotropic_diffusion
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import OneClassSVM

In [ ]:
def preprocess_input_pm(x):
    x = anisotropic_diffusion(x, niter=20, kappa=50, gamma=0.25)
    x = preprocess_input(x)
    return x

In [ ]:
DB_DIR = 'skindata3'
TRAIN_DIR = './fortraining/train'
TEST_DIR = './fortraining/val'

In [ ]:
split_folders.ratio(DB_DIR, output="fortraining", seed=16, ratio=(0.8,0.2)) # default values

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.losses import CategoricalCrossentropy


class WeightedCategoricalCrossentropy(CategoricalCrossentropy):
    
    def __init__(self, cost_mat, name='weighted_categorical_crossentropy', **kwargs):
        assert(cost_mat.ndim == 2)
        assert(cost_mat.shape[0] == cost_mat.shape[1])
        
        super().__init__(name=name, **kwargs)
        self.cost_mat = K.cast_to_floatx(cost_mat)
    
    def __call__(self, y_true, y_pred):
      
        return super().__call__(
            y_true=y_true,
            y_pred=y_pred,
            sample_weight=get_sample_weights(y_true, y_pred, self.cost_mat),
        )


def get_sample_weights(y_true, y_pred, cost_m):
    num_classes = len(cost_m)

    y_pred.shape.assert_has_rank(2)
    y_pred.shape[1].assert_is_compatible_with(num_classes)
    y_pred.shape.assert_is_compatible_with(y_true.shape)

    y_pred = K.one_hot(K.argmax(y_pred), num_classes)

    y_true_nk1 = K.expand_dims(y_true, 2)
    y_pred_n1k = K.expand_dims(y_pred, 1)
    cost_m_1kk = K.expand_dims(cost_m, 0)

    sample_weights_nkk = cost_m_1kk * y_true_nk1 * y_pred_n1k
    sample_weights_n = K.sum(sample_weights_nkk, axis=[1, 2])

    return sample_weights_n

In [ ]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        #yield (batch_x,[batch_y[:,i] for i in range(5)])
        yield (batch_x,[batch_y, batch_y, batch_y])

In [ ]:
class Adam_dlr(optimizers.Optimizer):

    """Adam optimizer.
    Default parameters follow those provided in the original paper.
    # Arguments
        split_1: split layer 1
        split_2: split layer 2
        lr: float >= 0. List of Learning rates. [Early layers, Middle layers, Final Layers]
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
    """

    def __init__(self, split_1, split_2, lr=[1e-7, 1e-4, 1e-2], beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False, **kwargs):
        super(Adam_dlr, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # Extracting name of the split layers
            #self.split_1 = split_1.weights[0].name
            #self.split_2 = split_2.weights[0].name
            self.split_1 = split_1
            self.split_2 = split_2
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad

    @keras.optimizers.interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats
        
        # Setting lr of the initial layers
        lr_grp = lr_t[0]
        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            
            # Updating lr when the split layer is encountered
            if p.name == self.split_1:
                lr_grp = lr_t[1]
            if p.name == self.split_2:
                lr_grp = lr_t[2]
                
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - lr_grp * m_t / (K.sqrt(vhat_t) + self.epsilon) # Using updated lr
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - lr_grp * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
#         print('Optimizer LR: ', K.get_value(self.lr))
#         print()
        config = {
                  'lr': (K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad}
        base_config = super(Adam_dlr, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def mergeDicts(dict1,dict2):
    keys = set(dict1).union(dict2)
    no = []
    return dict((k, dict1.get(k, no) + dict2.get(k, no)) for k in keys)

def plot_training(history):
  acc = history['categorical_accuracy']
  val_acc = history['val_categorical_accuracy']
  loss = history['loss']
  val_loss = history['val_loss']
  epochs = range(len(acc))
  
  plt.plot(epochs, acc)
  plt.plot(epochs, val_acc)
  plt.legend(['train', 'validation'], loc='upper right')
  plt.title('Training and validation accuracy')
  
  plt.figure()
  plt.plot(epochs, loss)
  plt.plot(epochs, val_loss)
  plt.title('Training and validation loss')
  plt.legend(['train', 'validation'], loc='upper right')
  plt.show()

def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: float(majority/count) for cls, count in counter.items()}

train_datagen = None
validation_datagen = None
train_generator = None
validation_generator = None
nb_train_samples = None
nb_validation_samples = None
train_data = None
train_labels = None
val_data = None
val_labels = None

def initNoneDataGenerators(shuffle_train=True):
    global train_datagen, validation_datagen, train_generator, validation_generator, nb_train_samples, nb_validation_samples
    # prepare data augmentation configuration
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=shuffle_train)

    validation_generator = validation_datagen.flow_from_directory(
        TEST_DIR,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    nb_train_samples = train_generator.samples
    nb_validation_samples = validation_generator.samples

def initDataGenerators(factor=1):
    global train_datagen, validation_datagen, train_generator, validation_generator, nb_train_samples, nb_validation_samples
    # prepare data augmentation configuration
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=160*factor,
        width_shift_range=0.01*factor,
        height_shift_range=0.01*factor,
        shear_range=0.15*factor,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='reflect')

    validation_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        #rotation_range=30,
        #width_shift_range=0.01,
        #height_shift_range=0.01,
        #shear_range=0.01,
        #zoom_range=0.1,
        #horizontal_flip=True,
        fill_mode='reflect')

    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

    validation_generator = validation_datagen.flow_from_directory(
        TEST_DIR,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    nb_train_samples = train_generator.samples
    nb_validation_samples = validation_generator.samples
    
def Read_Directory(path):
    data=[]
    labels=[]
    for class_ in os.listdir(path):
        if not class_.startswith('.'):
            dat = [os.path.join(path, class_, img) for img in os.listdir(os.path.join(path, class_)) if not img.startswith('.')]
            lab = [class_ for i in os.listdir(os.path.join(path, class_)) if not i.startswith('.')]
            labels = labels+lab
            #print(dat)
            data = data + dat

    data = np.array(data)
    labels = np.array(labels)
    return data, labels

def initCustomDataGenerators(factor=1, shuffle_train=True):
    global train_data, train_labels, val_data, val_labels, train_datagen, validation_datagen, train_generator, validation_generator, nb_train_samples, nb_validation_samples
    # prepare data augmentation configuration
    
    train_data, train_labels = Read_Directory(TRAIN_DIR)
    val_data, val_labels = Read_Directory(TEST_DIR)

    print('Training samples: {}'.format(len(train_data)))
    print('Validation samples: {}'.format(len(val_data)))

    train_datagen = FileDataGen(
        preprocessing_function=preprocess_input,
        aug_mode = 'StrongAug')

    validation_datagen = FileDataGen(
        preprocessing_function=preprocess_input,
        fill_mode='reflect')

    train_generator = train_datagen.flow_from_filelist(
        train_data, 
        train_labels,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=shuffle_train)

    validation_generator = validation_datagen.flow_from_filelist(
        val_data,
        val_labels,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    nb_train_samples = len(train_data)
    nb_validation_samples = len(val_data)
    
    print(nb_train_samples,nb_validation_samples)
    
def finetune_model(
    tune_level = 0, 
    lr = 0.001, 
    factor = 1, 
    verbose = 0, 
    callbacks = [], 
    epochs = 10, 
    spe_factor = 1, 
    loss = "binary_crossentropy", 
    gen = 'default', 
    split_1 = None,
    split_2 = None,
    radam_wu_steps = None,
    optimizer = 'adam'):
    
    if gen == 'default':
        initDataGenerators(factor)
    elif gen == 'custom':
        initCustomDataGenerators(factor)
    else:
        initNoneDataGenerators()
    
    K.set_learning_phase(1)
    
    p = round((1-tune_level)*len(model.layers))

    for layer in model.layers[:p]:
        layer.trainable = False
    for layer in model.layers[p:]:
        layer.trainable = True
        
    if tune_level == 0:
        for layer in model.layers[-CUSTOM_LAYERS_NUM:]:
            layer.trainable = True
        for layer in model.layers[:-CUSTOM_LAYERS_NUM]:
            layer.trainable = False
    
    if optimizer == 'adam':
        model.compile(optimizer=Adam(lr), loss=loss, metrics=[categorical_accuracy])
    elif optimizer == 'sgd':
        model.compile(optimizer=SGD(lr=lr, momentum=0.9, nesterov=True), loss=loss, metrics=[categorical_accuracy])
    elif optimizer == 'radam':
        model.compile(optimizer=Lookahead(RAdam(learning_rate=lr)), loss=loss, metrics=[categorical_accuracy])
    else:
        model.compile(optimizer=Adam_dlr(split_1, split_2, lr), loss=loss, metrics=[categorical_accuracy])
    #model.compile(optimizer=Nadam(lr), loss=loss, metrics=[categorical_accuracy])
    #model.compile(optimizer=SGD(lr=lr, momentum=0.9, decay=0.00001, nesterov=True), loss=loss, metrics=[categorical_accuracy])
    #model.compile(optimizer=Adadelta(), loss=loss, metrics=[categorical_accuracy])
    
    history = model.fit_generator(
        #generator_wrapper(train_generator),
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size * spe_factor,
        epochs=epochs,
        #validation_data=generator_wrapper(validation_generator),
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        class_weight=class_weight,
        callbacks=callbacks,
        verbose=verbose)
                  
    #plot_training(history)
    K.set_learning_phase(0)
    
    return history

def error_class_rate(gen):
    gen.reset()
    
#     predz = model.predict_generator(gen, steps=len(gen))
#     y_pred = np.argmax(preds, axis=1)
#     #print(confusion_matrix(gen.classes, y_pred))
#     classification_report(gen.classes, y_pred, output_dict = True)
    
    preds = model.predict_generator(gen, steps = len(gen))
    y_pred = np.argmax(preds, axis=1)
    #print(y_pred)
    y_true = gen.classes
    y_err = {0:0,1:0,2:0,3:0,4:0,5:0}
    err_num = 0
    for i, pred in enumerate(y_pred):
        if pred != y_true[i]:
            y_err[y_true[i]] = y_err[y_true[i]] + 1
            err_num = err_num + 1
            
    return {'y_err': y_err, 'er': err_num/len(y_true), 'err_num': err_num}

def miss_error_class_rate(gen):
    gen.reset()
    preds = model.predict_generator(gen, steps = len(gen))
    y_pred = np.argmax(preds, axis=1)
    #print(y_pred)
    y_true = gen.classes
    y_err = {0:0,1:0,2:0,3:0,4:0,5:0}
    err_num = 0
    for i, pred in enumerate(y_pred):
        if pred != y_true[i]:
            y_err[pred] = y_err[pred] + 1
            err_num = err_num + 1
            
    return {'y_err': y_err, 'er': err_num/len(y_true), 'err_num': err_num}

def evaluate_dynamic_weights(gen, miss_error = False):
    global class_weight
    
    ecr = None
#     if miss_error:
#         ecr = miss_error_class_rate(gen)
#     else:
#         ecr = error_class_rate(gen)

    gen.reset()
    preds = model.predict_generator(gen, steps=len(gen))
    y_pred = np.argmax(preds, axis=1)
    #print(confusion_matrix(gen.classes, y_pred))
    class_weight = {0:0,1:0,2:0,3:0,4:0,5:0}
    metrs = classification_report(gen.classes, y_pred, output_dict = True)
    for cls, metr in class_weight.items():
        #print(cls)
        #print(class_weight[cls])
        class_weight[cls] = metrs[str(cls)]['f1-score']
    #class_weight = classification_report(gen.classes, y_pred, output_dict = True)[0:5]
#     for cls, metr in class_weight.items():
#         print(cls,metr['f1-score'])
    #class_weight = {int(cls): float(1-metr['f1-score']) for cls, metr in class_weight.items()}
    
    #minerr = min(y_err.values())
    #class_weight = {cls: float(count/ecr['err_num'])*6.0 for cls, count in ecr['y_err'].items()}
    
    #print('Error classes', y_err)
    #majority = max(counter.values())
    #return  {cls: float(majority/count) for cls, count in counter.items()}
    #class_weight = get_class_weights(y_err)
#     for i in range(5):
#         if not i in class_weight:
#             class_weight[i] = 1
    print(class_weight)

    #return np.mean(class_weight.values())
    return 0
    #print('New weights', class_weight)
    
def finetune_with_adj_weights(tune_level = 0, 
                               lr = 0.001, 
                               factor = 1, 
                               callbacks = [], 
                               epochs = 10,
                               miss_error = False,
                               verbose = 1,
                               spe_factor = 1, 
                               loss = "binary_crossentropy", 
                               gen = 'default',
                               epochs_per_super = 1,
                               save_models = True,
                               optimizer = 'adam'):
    trainhist = {}
    for i in range(epochs):
        print('_____________ SUPER EPOCH #',i)
        newhist = finetune_model(tune_level, lr, factor, verbose, callbacks, epochs_per_super, spe_factor, loss, gen, optimizer)
        datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
        generator = datagen.flow_from_directory(
            TRAIN_DIR,
            target_size=(img_height, img_width),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False)
        er = evaluate_dynamic_weights(generator, miss_error)
        if save_models:
            model.save('model_se'+str(i)+'_er'+str(np.round(er, 2))+'.hdf5')
        
        trainhist = mergeDicts(trainhist,newhist.history)
        print('_____________ error rate = ', er)
    plot_training(trainhist)
    if save_models:
        plt.savefig('models_hist.png')
    #print(class_weight)
    
def confMatrix(gen):
    gen.reset()
    Y_pred = model.predict_generator(gen, steps=len(gen))
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    #print(confusion_matrix(gen.classes, y_pred))
    seaborn.heatmap(confusion_matrix(gen.classes, y_pred))
    print('Classification Report')
    target_names = ['AD', 'CPOX', 'CS', 'HIVES', 'NE', 'P', 'RM']
    print(classification_report(gen.classes, y_pred, target_names=target_names))
    
def prepare_image(image):
    # if the image mode is not RGB, convert it
    if image.mode != "RGB":
        image = image.convert("RGB")

    # resize the input image and preprocess it
    image = image.resize((img_width, img_height))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image
    
def predict_tta(datagen, model, samples, n_examples):
    # convert image into dataset
    # samples = np.expand_dims(image, 0)
    # prepare iterator
    it = datagen.flow(samples, batch_size=n_examples, shuffle=False)
    # make predictions for each augmented image
    yhats = model.predict_generator(it, steps=n_examples, verbose=0)
    # sum across predictions
    mean_probs = np.mean(yhats, axis=0)
    # argmax across classes
    return mean_probs

In [ ]:
K.set_learning_phase(0)

img_width, img_height = 299, 299

base_model = InceptionV3(weights='imagenet', include_top=False)

batch_size = 16

In [ ]:
#x1 = base_model.layers[228].output
x1 = model.layers[228].output
x1 = GlobalAveragePooling2D()(x1)
#feature_extractor = Model(inputs=base_model.input, outputs=x1)
feature_extractor = Model(inputs=model.input, outputs=x1)

batch_size = 1
initCustomDataGenerators(1, shuffle_train=False)
#initNoneDataGenerators(shuffle_train=False)
aug_n = 1

y = np.tile(train_generator.classes,aug_n)
X = feature_extractor.predict_generator(train_generator, steps = nb_train_samples // batch_size * aug_n)

In [ ]:
#clf = RandomForestClassifier(n_estimators=1000, oob_score=True, min_samples_split=10, criterion='gini', bootstrap=True, random_state=5)
clf = SVC(kernel='linear', C=5, decision_function_shape='ovo', probability=True)

clf.fit(X, y)
#clf = OneClassSVM(kernel='linear').fit(X)

#predict_proba

In [ ]:
# r = 0
# rc = clf.predict(X)
# for c in rc:
#     if c<0:
#         r = r + 1
        
# print(r/len(rc))

In [ ]:
validation_generator.reset()
y_test = validation_generator.classes
X_test = feature_extractor.predict_generator(validation_generator, steps = nb_validation_samples // batch_size)
y_pred = clf.predict(X_test)

seaborn.heatmap(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
extra_validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

extra_validation_generator = extra_validation_datagen.flow_from_directory(
    'valid',
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)
extra_nb_validation_samples = extra_validation_generator.samples
#validation_generator.reset()
y_test = extra_validation_generator.classes
X_test = feature_extractor.predict_generator(extra_validation_generator, steps = extra_nb_validation_samples // batch_size)
y_pred = clf.predict(X_test)

seaborn.heatmap(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#model.summary()
for i,l in enumerate(base_model.layers):
    print(i,l.name)

In [ ]:
CUSTOM_LAYERS_NUM = 3

# add a global spatial average pooling layer
#x = base_model.layers[196].output
#x = base_model.layers[164].output
#x = base_model.output
# x = base_model.layers[228].output
# x = GlobalAveragePooling2D()(x)
# # x = Dropout(0.3)(x)
# x = Dense(128, activation='elu')(x)
# #x = Dense(30, activation='elu')(x)
# predictions = Dense(6, activation='sigmoid')(x)
x1 = base_model.layers[228].output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(128, activation='elu')(x1)
#x1 = Dense(6, activation='sigmoid')(x1)

# x2 = base_model.layers[196].output
# x2 = GlobalMaxPooling2D()(x2)
# x2 = Dense(128, activation='elu')(x2)
# #x2 = Dense(6, activation='sigmoid')(x2)

# x3 = base_model.layers[228].output
# x3 = GlobalMaxPooling2D()(x3)
# x3 = Dense(128, activation='elu')(x3)
# #x3 = Dense(6, activation='sigmoid')(x3)

# x = Minimum()([x1,x2,x3])
#predictions = Dense(6, activation='linear', kernel_regularizer=l2(0.01))(x1)
predictions = Dense(7, activation='sigmoid')(x1)


model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
#model.summary()
for i,l in enumerate(model.layers):
    print(i,l.name)

In [ ]:
initCustomDataGenerators(1)
#class_weight = get_class_weights(train_generator.classes)
class_weight = {0:1, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1}
print(class_weight)

In [ ]:
x_batch, y_batch = next(train_generator)

plt.figure(figsize=(14, 18))
for k, (img, lbl) in enumerate(zip(x_batch, y_batch)):
    plt.subplot(6, 5, k+1)
    plt.imshow((img + 1) / 2)
    plt.axis('off')

In [ ]:
print(dir(train_generator))
print(train_generator.index_array)

In [ ]:
class_weight

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3, restore_best_weights=True)
rlp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=0, verbose=1, mode='auto', min_delta=0.01, cooldown=2, min_lr=0.000001)
histplot = PlotLossesCallback(plot_extrema=False)

# finetune_with_adj_weights( tune_level = 0, 
#                            epochs_per_super = 2,
#                            lr = 0.001, 
#                            factor = 1,
#                            verbose = 1,
#                            callbacks = [], 
#                            epochs = 5,
#                            spe_factor = 1,
#                            loss = losses.binary_crossentropy, 
#                            gen = 'custom', 
#                            optimizer = 'adam')

finetune_model(tune_level = 0, 
               lr = 0.001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 40,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom',
               optimizer = 'adam')

In [ ]:
model.save('top_new_3_1.hdf5')

In [ ]:
model = klm('top_new_3_1.hdf5')

In [ ]:
finetune_with_adj_weights( tune_level = 0, 
                           epochs_per_super = 2,
                           lr = 0.001, 
                           factor = 1,
                           verbose = 1,
                           callbacks = [], 
                           epochs = 5,
                           spe_factor = 1,
                           loss = losses.binary_crossentropy, 
                           gen = 'custom', 
                           optimizer = 'adam')

In [ ]:
finetune_with_adj_weights( tune_level = 0, 
                           epochs_per_super = 2,
                           lr = 0.001, 
                           factor = 1,
                           verbose = 1,
                           callbacks = [], 
                           epochs = 4,
                           spe_factor = 1,
                           loss = losses.binary_crossentropy, 
                           gen = 'custom', 
                           optimizer = 'adam')

In [ ]:
class_weight = {0:1, 1:1, 2:1, 3:1, 4:1, 5:1}

In [ ]:
finetune_model(tune_level = 0.1, 
               lr = 0.001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 40,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom', 
               optimizer = 'adam')

In [ ]:
x = model.layers[-3].output
#x = GlobalAveragePooling2D()(x)
# x = Dropout(0.3)(x)
x = Dense(128, activation='elu')(x)
#x = Dense(30, activation='elu')(x)
predictions = Dense(6, activation='sigmoid')(x)

model = Model(inputs=model.input, outputs=predictions)

In [ ]:
finetune_model(tune_level = 0, 
               lr = 0.001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 40,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom',
               optimizer = 'adam')

In [ ]:
finetune_model(tune_level = 0.2, 
               lr = 0.0001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 40,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom', 
               optimizer = 'adam')

In [ ]:
x = model.layers[-3].output
#x = GlobalAveragePooling2D()(x)
# x = Dropout(0.3)(x)
x = Dense(128, activation='elu')(x)
#x = Dense(30, activation='elu')(x)
predictions = Dense(6, activation='sigmoid')(x)

model = Model(inputs=model.input, outputs=predictions)

In [ ]:
finetune_model(tune_level = 0.4, 
               lr = 0.0001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 40,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom',
               optimizer = 'adam')

In [ ]:
x = model.layers[-3].output
#x = GlobalAveragePooling2D()(x)
# x = Dropout(0.3)(x)
x = Dense(128, activation='elu')(x)
#x = Dense(30, activation='elu')(x)
predictions = Dense(6, activation='sigmoid')(x)

model = Model(inputs=model.input, outputs=predictions)

finetune_model(tune_level = 0.1, 
               lr = 0.001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 4,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom', 
               optimizer = 'adam')

In [ ]:
finetune_model(tune_level = 0, 
               lr = 0.001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 40,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom',
               optimizer = 'adam')

In [ ]:
model.save('top_new_1.hdf5')

In [ ]:
import math
def exp_decay(lr = 0.1, k = 0.1):
    return lambda epoch: lr * math.exp(-k*epoch)   

In [ ]:
lrs = LearningRateScheduler(exp_decay(lr = 0.001, k = 0.1))

finetune_model(tune_level = 0, 
               lr = 0.001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es,lrs], 
               epochs = 20,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom', 
               optimizer = 'sgd')

In [ ]:
model.save('top_new_2.hdf5')

In [ ]:
finetune_model(tune_level = 0, 
               lr = 0.0001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 40,
               spe_factor = 1,
               loss = losses.binary_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom',
               optimizer = 'adam')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3, restore_best_weights=True)
rlp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=0, verbose=1, mode='auto', min_delta=0.01, cooldown=2, min_lr=0.000001)
histplot = PlotLossesCallback(plot_extrema=False)

finetune_model(tune_level = 0.1, 
               lr = 0.0001, 
               factor = 1, 
               verbose = 1,
               callbacks = [es], 
               epochs = 20,
               spe_factor = 1,
               #loss = losses.categorical_crossentropy, 
               #loss = WeightedCategoricalCrossentropy(np.array([[0,1,2,2,2,1],[1,0,1,1.5,1,2],[2,1,0,1,1,1],[2,1.5,1,0,1.5,1],[2,1,1,1.5,0,1],[1,2,1,1,1,0]])),
               gen = 'custom', 
               optimizer = 'adam')

In [ ]:
model.save('fine_new_8_best.hdf5')

In [ ]:
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
    
confMatrix(validation_generator)

In [ ]:
model = klm('fine_new_8_best.hdf5')

In [ ]:
len(X)

In [ ]:
x1 = model.layers[228].output
x1 = GlobalAveragePooling2D()(x1)
#feature_extractor = Model(inputs=base_model.input, outputs=x1)
feature_extractor = Model(inputs=model.input, outputs=x1)

initCustomDataGenerators(1)
Xtr = feature_extractor.predict_generator(train_generator, steps = nb_train_samples // batch_size)
Xte = feature_extractor.predict_generator(validation_generator, steps = nb_validation_samples // batch_size)
X = np.concatenate((Xtr, Xte), axis=0)

In [ ]:
ae_input = Input(shape=(768,))
x = Dense(10, activation='elu', activity_regularizer=l1(10e-5))(ae_input)
# x = Dense(128, activation='elu')(ae_input)
# x = Dense(64, activation='elu')(x)
# x = Dense(2, activation='elu')(x)
# x = Dense(64, activation='elu')(x)
# x = Dense(128, activation='elu')(x)
ae_output = Dense(768, activation='sigmoid')(x)
autoencoder = Model(inputs=ae_input, outputs=ae_output)

In [ ]:
#initCustomDataGenerators(1)
    
K.set_learning_phase(1)

# for layer in autoencoder.layers[:-2]:
#     layer.trainable = False
for layer in autoencoder.layers:
    layer.trainable = True
    
autoencoder.compile(optimizer=Adam(0.0001), loss=losses.binary_crossentropy, metrics=[])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1, restore_best_weights=True)

history = autoencoder.fit(
    X[:600],
    X[:600],
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=100,
    validation_data=(X[600:], X[600:]),
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[],
    verbose=1)

#plot_training(history)
K.set_learning_phase(0)

In [ ]:
K.set_learning_phase(1)
history = autoencoder.fit(
    X[:600],
    X[:600],
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=50,
    validation_data=(X[600:], X[600:]),
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[es],
    verbose=1)

#plot_training(history)
K.set_learning_phase(0)

In [ ]:
autoencoder.evaluate(X[600:], X[600:], steps=nb_validation_samples)

In [ ]:
autoencoder.evaluate(X[:600], X[:600], steps=nb_train_samples)

In [ ]:
xv_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
xv_generator = xv_datagen.flow_from_directory(
    'valid',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Xv = feature_extractor.predict_generator(xv_generator, steps = xv_generator.samples // batch_size)

autoencoder.evaluate(Xv, Xv, steps=xv_generator.samples)

In [ ]:
xwrong_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
xwrong_generator = xwrong_datagen.flow_from_directory(
    'wrongdata/ne',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Xwrong = feature_extractor.predict_generator(xwrong_generator, steps = xwrong_generator.samples // batch_size)

autoencoder.evaluate(Xwrong, Xwrong, steps=xwrong_generator.samples)

In [ ]:
#from keras.models import load_model as klm

# model1 = klm('model_se4_er0.02.hdf5')
# model2 = klm('fine_new_4.hdf5')
# model3 = klm('fine_new_5.hdf5')

K.set_learning_phase(0)
#model = klm('./cnns/model-finetune-v1-04-0.05.hdf5')
#model = klm('./finetunemodel_adj_w/model_se2_er0.14.hdf5')

# for layer in model.layers:
#     layer.trainable = False

# validation_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     #rotation_range=30,
#     #width_shift_range=0.01,
#     #height_shift_range=0.01,
#     #shear_range=0.01,
#     #zoom_range=0.1,
#     #horizontal_flip=True,
#     fill_mode='reflect')

# validation_generator = validation_datagen.flow_from_directory(
#     'valid',
#     target_size=(img_width, img_height),
#     batch_size=16,
#     class_mode='categorical',
#     shuffle=False)

tdg = ImageDataGenerator(
    rotation_range=60,
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    #shear_range=0.2,
    #zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

n_examples = 10

score = 0

test_db = glob('./valid/atopic_dermatitis/*.*') + glob('./valid/hives/*.*') + glob('./valid/nummular_eczema/*.*')
test_db_classes = [0]*len(glob('./valid/atopic_dermatitis/*.*')) + [3]*len(glob('./valid/hives/*.*')) + [4]*len(glob('./valid/nummular_eczema/*.*'))

for i, imurl in enumerate(test_db):
    image = prepare_image(PIL.Image.open(imurl))
    pred = predict_tta(tdg, model, image, n_examples)
#     pred1 = predict_tta(tdg, model1, image, n_examples)
#     pred2 = predict_tta(tdg, model2, image, n_examples)
#     pred3 = predict_tta(tdg, model3, image, n_examples)
#     pred = np.average([pred1, pred2, pred3], axis=0)
    print(imurl, np.round(pred, 2))
    #print(test_db_classes[i], np.equal(np.argmax(pred),test_db_classes[i]))
    score = score + np.equal(np.argmax(pred),test_db_classes[i])
    
print('tta score ', score/len(test_db))

tdg = ImageDataGenerator()
n_examples = 1
score = 0

for i, imurl in enumerate(test_db):
    image = prepare_image(PIL.Image.open(imurl))
    pred = predict_tta(tdg, model, image, n_examples)
#     pred1 = predict_tta(tdg, model1, image, n_examples)
#     pred2 = predict_tta(tdg, model2, image, n_examples)
#     pred3 = predict_tta(tdg, model3, image, n_examples)
#     pred = np.average([pred1, pred2, pred3], axis=0)
    print(imurl, np.round(pred, 2))
    #print(test_db_classes[i], np.equal(np.argmax(pred),test_db_classes[i]))
    score = score + np.equal(np.argmax(pred),test_db_classes[i])
    
print('no-tta score ', score/len(test_db))

# print(validation_generator.filenames)
# print(model.evaluate_generator(validation_generator))
# preds = model.predict_generator(validation_generator)
# print(np.round(preds, 2))
# #y_pred = np.round(preds, 2)
# y_pred = np.argmax(preds, axis=1)
# print(y_pred)
# for i, pred in enumerate(y_pred):
#     print(i,pred)
# y_true = validation_generator.classes
# print(y_pred,y_true)